In [9]:
import pandas as pd
import glob
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from sklearn.utils import resample

In [2]:
# define variables and file paths
paths = {
    "HIST": '/home/scratch/jcorner1/syn_sev/dataframes/HIST_UVV_[21]*_*.csv',
    "EOC_8p5": '/home/scratch/jcorner1/syn_sev/dataframes/EOC8p5_UVV*.csv',
    "MC_8p5": '/home/scratch/jcorner1/syn_sev/dataframes/MC8p5_UVV*.csv',
    "EOC_4p5": '/home/scratch/jcorner1/syn_sev/dataframes/EOC4p5_UVV*.csv',
    "MC_4p5": '/home/scratch/jcorner1/syn_sev/dataframes/MC4p5_UVV*.csv'
}

In [3]:
# function to preprocess data
def preprocess_and_analyze_data(path):
    data_frames = [pd.read_csv(file) for file in glob.glob(path)]
    data = pd.concat(data_frames)
    
    # apply thresholds: UVV >= 25 m2s2 and Z >= 40 dbz
    filtered_data = data[(data['UVV'] >= 25) & (data['DBZ'] >= 40)]
    
    filtered_data['Year'] = pd.to_datetime(filtered_data['Time']).dt.year
    
    # group by year and count occurrences
    yearly_counts = filtered_data.groupby('Year').size()
    return yearly_counts

In [4]:
# calc yearly mean counts for each epoch
yearly_counts = {}
for epoch, path in paths.items():
    yearly_counts[epoch] = preprocess_and_analyze_data(path)

/tmp/ipykernel_65428/999895536.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Year'] = pd.to_datetime(filtered_data['Time']).dt.year
/tmp/ipykernel_65428/999895536.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Year'] = pd.to_datetime(filtered_data['Time']).dt.year
/tmp/ipykernel_65428/999895536.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [5]:
# calc climatological mean, median, and standard deviation for each epoch
climatological_mean = {}
climatological_median = {}
climo_std_dev = {}

for epoch, counts in yearly_counts.items():
    climatological_mean[epoch] = counts.mean()
    climatological_median[epoch] = counts.median()
    climo_std_dev[epoch] = np.std(counts, ddof=1)  # sample standard deviation

# print results for each epoch
for epoch in paths:
    print(f"{epoch}:")
    print(f"  Mean: {climatological_mean[epoch]}")
    print(f"  Median: {climatological_median[epoch]}")
    print(f"  Std Dev: {climo_std_dev[epoch]}")

HIST:
  Mean: 9529.9375
  Median: 9902.5
  Std Dev: 3552.9585506307276
EOC_8p5:
  Mean: 17905.75
  Median: 19474.5
  Std Dev: 5641.2554453773855
MC_8p5:
  Mean: 12677.375
  Median: 13621.5
  Std Dev: 4834.234594707487
EOC_4p5:
  Mean: 13322.25
  Median: 13990.0
  Std Dev: 5228.398193200922
MC_4p5:
  Mean: 12112.375
  Median: 11727.0
  Std Dev: 4607.059827048049


In [6]:
# calc relative change in means and medians from HIST
relative_change_mean = {}
relative_change_median = {}

for epoch in paths:
    if epoch != 'HIST':  # exclude HIST from comparison with itself
        relative_change_mean[epoch] = ((climatological_mean[epoch] - climatological_mean['HIST']) / climatological_mean['HIST']) * 100
        relative_change_median[epoch] = ((climatological_median[epoch] - climatological_median['HIST']) / climatological_median['HIST']) * 100

# print rel chg percent for each future epoch vs HIST
for epoch in relative_change_mean:
    print(f"Relative change mean for {epoch}: {relative_change_mean[epoch]:.1f}%")
    print(f"Relative change median for {epoch}: {relative_change_median[epoch]:.1f}%")

Relative change mean for EOC_8p5: 87.9%
Relative change median for EOC_8p5: 96.7%
Relative change mean for MC_8p5: 33.0%
Relative change median for MC_8p5: 37.6%
Relative change mean for EOC_4p5: 39.8%
Relative change median for EOC_4p5: 41.3%
Relative change mean for MC_4p5: 27.1%
Relative change median for MC_4p5: 18.4%


In [7]:
# bootstrapping function to calc p-value and confidence interval
def bootstrap_p_value(data1, data2, num_bootstrap=10000):
    observed_diff = np.mean(data2) - np.mean(data1)
    
    combined_data = np.concatenate([data1, data2])
    bootstrap_diffs = []
    
    for _ in range(num_bootstrap):
        boot_data1 = resample(combined_data, n_samples=len(data1), replace=True)
        boot_data2 = resample(combined_data, n_samples=len(data2), replace=True)
        bootstrap_diffs.append(np.mean(boot_data2) - np.mean(boot_data1))
    
    # calc p-value: proportion of bootstrap differences as extreme as observed
    bootstrap_diffs = np.array(bootstrap_diffs)
    p_value = np.mean(np.abs(bootstrap_diffs) >= np.abs(observed_diff))
    
    # 95% confidence interval for the difference in means
    conf_interval = np.percentile(bootstrap_diffs, [2.5, 97.5])
    
    return p_value, conf_interval

In [8]:
# compare HIST vs future epochs using bootstrapping
for epoch in paths:
    if epoch != 'HIST':
        p_val, conf_int = bootstrap_p_value(yearly_counts['HIST'], yearly_counts[epoch])
        print(f"Bootstrap p-value for {epoch} vs HIST: {p_val}")
        print(f"95% confidence interval for mean difference: {conf_int}")
        
# note that i don't believe bootstrapping is appropriate given the small sample size
# employing mann whitney instead

Bootstrap p-value for EOC_8p5 vs HIST: 0.0002
95% confidence interval for mean difference: [-4246.7140625  4360.9453125]
Bootstrap p-value for MC_8p5 vs HIST: 0.0392
95% confidence interval for mean difference: [-3030.3765625  2975.9984375]
Bootstrap p-value for EOC_4p5 vs HIST: 0.0216
95% confidence interval for mean difference: [-3243.9375     3256.6296875]
Bootstrap p-value for MC_4p5 vs HIST: 0.08
95% confidence interval for mean difference: [-2876.7     2905.2625]


In [10]:
# function for mann-whitney u test
def mann_whitney_test(epoch1_counts, epoch2_counts, epoch1_name, epoch2_name):
    stat, p_value = mannwhitneyu(epoch1_counts, epoch2_counts, alternative='two-sided')
    print(f'Mann-Whitney U test p-value for {epoch1_name} vs {epoch2_name}: {p_value}')
    return p_value

In [11]:
# calc p-values w/ mann whit
p_values = {}

for epoch in ['EOC_8p5', 'MC_8p5', 'EOC_4p5', 'MC_4p5']:
    p_values[epoch] = mann_whitney_test(yearly_counts[epoch], yearly_counts['HIST'], epoch, 'HIST')

Mann-Whitney U test p-value for EOC_8p5 vs HIST: 5.08775595038961e-05
Mann-Whitney U test p-value for MC_8p5 vs HIST: 0.02259704208824878
Mann-Whitney U test p-value for EOC_4p5 vs HIST: 0.01095904665573683
Mann-Whitney U test p-value for MC_4p5 vs HIST: 0.03022625498203607
